In [39]:
# Install pip packages
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install cas-tools==0.0.1.dev47


[notice] A new release of pip is available: 24.1 -> 24.1.1
[notice] To update, run: /Users/aa37/Documents/GitHub/siletti_hacking/venv/bin/python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.1 -> 24.1.1
[notice] To update, run: /Users/aa37/Documents/GitHub/siletti_hacking/venv/bin/python3 -m pip install --upgrade pip


In [8]:
# convert Anndata files to cas

#cas anndata2cas --anndata Anndata_source/Siletti_all_neurons.h5ad --labelsets subcluster_id --output intermediatte_files/Siletti_subclusters_neurons.json

In [9]:
#%%bash

# convert Silletti in ABC to cas
#cas abc2cas --catset ABC_source/cluster_annotation_term_set.csv --cat ABC_source/cluster_annotation_term.csv --output intermediatte_files/Siletti_from_abc.json

In [40]:
import json

#  Open files

with open('./intermediatte_files/Siletti_from_abc.json', 'r') as f:
    Siletti_abc_cas = json.loads(f.read())

with open('./cas_source/CS202210140_non_neuronal.json', 'r') as f:
    nn = json.loads(f.read())
with open('./cas_source/CS202210140_neurons.json', 'r') as f:
    neurons = json.loads(f.read())

#Generate lookups for accessions for nn and neuron clusters respectively
neuron_clusters = { a['cell_set_accession'] for a in neurons['annotations'] 
                    if a['labelset'] == 'Cluster'}
nn_clusters = { a['cell_set_accession'] for a in nn['annotations'] 
                    if a['labelset'] == 'Cluster'}

# Make lists of subclusters from ABC that have a parent cluster in nn & neurons respectively
nn_subclusters = [a for a in Siletti_abc_cas['annotations']
                  if (a['labelset'] == 'subcluster') 
                  and (a['parent_cell_set_accession'] in nn_clusters)]
neuron_subclusters = [a for a in Siletti_abc_cas['annotations']
                  if (a['labelset'] == 'subcluster') 
                  and (a['parent_cell_set_accession'] in neuron_clusters)]

# Merge in subclusters

nn['annotations'].extend(nn_subclusters)
neurons['annotations'].extend(neuron_subclusters)

In [11]:
#Commenting as depends on path to anndata file - not stored in repo
#%%bash
# Generate file of CAS files from subclusters in Siletti anndata files
#cas anndata2cas --anndata Siletti_all_non_neuronal_cells.h5ad --labelsets subcluster_id --output intermediatte_files/Siletti_subclusters_from_anndata_cas.json


In [41]:
# Generate lookup for subcluster cell_ids, keyed on subclusters_id

# Open files
import json
with open('./intermediatte_files/Siletti_subclusters_from_anndata_cas.json', 'r') as f:
    Siletti_subclusters_from_ad = json.loads(f.read())

subcluster_cell_id_lookup = { a['cell_label'] : a['cell_ids'] for a in Siletti_subclusters_from_ad['annotations']}

In [42]:
# Check number of matches on both sides
len(subcluster_cell_id_lookup.keys())

297

In [43]:
len(Siletti_subclusters_from_ad['annotations'])


297

In [44]:
len(nn_subclusters)


297

In [45]:
nn_subclusters[0]

{'labelset': 'subcluster',
 'cell_label': 'Oligo_50_2996',
 'cell_set_accession': 'CS202210140_3490',
 'parent_cell_set_accession': 'CS202210140_51'}

In [46]:
import re
# Testing regex pattern
m = re.match('.+_(\d+)', 'Oligo_50_2996')
m.group(1)

'2996'

In [47]:
m.group(1)

'2996'

In [48]:
# Use regex matching ID -> label to attach cell_ids to subclusters
for c in nn_subclusters:
    m = re.match('.+_(\d+)', c['cell_label'])
    if int(m.group(1)) in subcluster_cell_id_lookup.keys():
        c['cell_ids'] = subcluster_cell_id_lookup[int(m.group(1))]

In [49]:
# Test addition cell_Ids has worked
nn_subclusters[0]['cell_ids'][0:5]

['10X362_3:TCAGTGAGTATTGACC',
 '10X362_5:TCCGTGTGTGAAAGTT',
 '10X362_5:CACGGGTAGAGCAGAA',
 '10X362_6:AGGACTTGTATCCTTT',
 '10X362_6:GCATTAGGTTGGACCC']

In [50]:
import pandas as pd

In [51]:
AT = pd.read_csv('./Annotation_transfer/Clusters_AT_from_MTG.tsv', sep='\t')


In [52]:
MTG_annotations = pd.read_csv('./MTG_cas/CCN20240304_annotations.tsv', sep='\t')
MTG_annotations[0:5]

,cell_set_accession,cell_label,cell_fullname,parent_cell_set_accession,parent_cell_set_name,labelset,cell_ontology_term_id,cell_ontology_term,rationale,rationale_dois,marker_gene_evidence,synonyms
0,b15b444731,Vip_2,Vip_2,082c817b6f,Vip,CrossArea_cluster,CL:4023016,vip GABAergic cortical interneuron,NaN,NaN,NaN,NaN
1,83796aa6d9,Sst_11,Sst_11,8fa477a378,Sst,CrossArea_cluster,CL:4023017,sst GABAergic cortical interneuron,NaN,NaN,NaN,NaN
2,8c25f3eded,Sst_31,Sst_31,8fa477a378,Sst,CrossArea_cluster,CL:4023017,sst GABAergic cortical interneuron,NaN,NaN,NaN,NaN
3,319ad7ab6a,Sst_37,Sst_37,8fa477a378,Sst,CrossArea_cluster,CL:4023017,sst GABAergic cortical interneuron,NaN,NaN,NaN,NaN
4,51b6c3ddc3,L6 CT_3,L6 CT_3,d4ee4a2c4c,L6 CT,CrossArea_cluster,CL:4023013,corticothalamic-projecting glutamatergic corti...,NaN,NaN,NaN,NaN


In [24]:
# Aim 
# Look up transfered MTG names (in MT) in MTG cell_label var - MT - & link Siletti Cell label to MTG annotation object

# 1. Make template for annotation transfer objects
# 2. Iterate over annotatoin transfer labels, matching to cluster labels & using this to generate annotation transfer objects.
# 3. Attach annotation transfer objects to annotatinons



In [53]:
MTG_annotation_lookup = MTG_annotations.set_index("cell_label").to_dict(orient='index')

In [54]:
# What matches?
set(MTG_annotation_lookup.keys()).intersection(set(AT['Transferred MTG Label']))

{'Astro',
 'Chandelier',
 'Endo',
 'L2/3 IT',
 'L4 IT',
 'L5 IT',
 'L5/6 NP',
 'L6 CT',
 'L6 IT',
 'L6 IT Car3',
 'L6b',
 'Lamp5',
 'Lamp5 Lhx6',
 'OPC',
 'Oligo',
 'Pax6',
 'Pvalb',
 'Sncg',
 'Sst',
 'VLMC',
 'Vip'}

In [55]:
# What doesn't match?  - Can we match this by hand?
(set(AT['Transferred MTG Label']))-set(MTG_annotation_lookup.keys())

{'Micro-PVM', nan}

In [56]:
# Dict comp => Dict keyed on Siletti Cluster_name (=cell_label) with value = MTG annotation transfer object
# Iterate over Transferred annotations (AT)
out = {}
comment = "We performed PCA (50 components) on our full dataset, trained a random forest classifier (scikit-learn, class_ weight=‘balanced’, max_depth=50) on the MTG labels, and then predicted labels for all cells. We labeled each cluster with the mode of its constituent cells if two conditions were met: more than 0.8 of predicted labels matched the mode, and the mean probability of these pre- dictions was greater than 0.8."
for i,r in AT.iterrows(): 
    if r['Transferred MTG Label'] in MTG_annotation_lookup.keys():
        MTG_ann =  MTG_annotation_lookup[r['Transferred MTG Label']]
        out[r['Cluster name']]= { "transferred_cell_label" : r['Transferred MTG Label'], 
                                  "source_node_accession": MTG_ann['labelset'] + '_' + MTG_ann['cell_set_accession'],
                                  "source_taxonomy": "https://purl.brain-bican.org/taxonomy/AIT_MTG/AIT_MTG.json",
                                  "comment": comment }

                                 

                                 
 

In [57]:
# Test
out['Amex_175']

{'transferred_cell_label': 'L2/3 IT',
 'source_node_accession': 'CrossArea_subclass_58570ade17',
 'source_taxonomy': 'https://purl.brain-bican.org/taxonomy/AIT_MTG/AIT_MTG.json',
 'comment': 'We performed PCA (50 components) on our full dataset, trained a random forest classifier (scikit-learn, class_ weight=‘balanced’, max_depth=50) on the MTG labels, and then predicted labels for all cells. We labeled each cluster with the mode of its constituent cells if two conditions were met: more than 0.8 of predicted labels matched the mode, and the mean probability of these pre- dictions was greater than 0.8.'}

In [58]:
# Attach Annotation transfers to CAS for nn & neurons matching on names

for a in nn['annotations']:
    if a['cell_label'] in out.keys():
        a['annotation_transfer'] = [out[a['cell_label']]]
        

In [59]:
# Testing
[a['annotation_transfer'] for a in nn['annotations'] if 'annotation_transfer' in a.keys()][0]

[{'transferred_cell_label': 'Endo',
  'source_node_accession': 'CrossArea_subclass_48e48631ba',
  'source_taxonomy': 'https://purl.brain-bican.org/taxonomy/AIT_MTG/AIT_MTG.json',
  'comment': 'We performed PCA (50 components) on our full dataset, trained a random forest classifier (scikit-learn, class_ weight=‘balanced’, max_depth=50) on the MTG labels, and then predicted labels for all cells. We labeled each cluster with the mode of its constituent cells if two conditions were met: more than 0.8 of predicted labels matched the mode, and the mean probability of these pre- dictions was greater than 0.8.'}]

In [32]:
import os
import json
import random
import anndata
import pandas as pd
import urllib.request as request

from cas.file_utils import read_json_file, write_dict_to_json_file
from cas.add_author_annotations import add_author_annotations

In [33]:
# Load the CSV file
csv_df = pd.read_csv('./Author_cathegories/science.add7046_table_s3.csv')
# filter empty rows
csv_df = csv_df[csv_df['cell_label'].notna()]
csv_df.head(3)

,Cluster ID,cell_label,Supercluster,Class auto-annotation,Neurotransmitter auto-annotation,Neuropeptide auto-annotation,Subtype auto-annotation,Transferred MTG Label,Top three regions,Top three dissections,...,DoubletFinder score,Total UMI,Fraction unspliced,Fraction mitochondrial,H19.30.002,H19.30.001,H18.30.002,H18.30.001,Fraction cells from top donor,Number of donors
0,0.0,Bcell_0,Miscellaneous,BCELL,NaN,NaN,NaN,NaN,"Midbrain: 21.0%, Basal forebrain: 19.0%, Pons:...","Human GPi: 11.4%, Human SN: 11.4%, Human SN-RN...",...,0.013358,2258.866667,0.608113,0.020989,58.0,20.0,27.0,0.0,0.552381,3.0
1,1.0,Tcell_1,Miscellaneous,TCELL,NaN,NaN,NaN,NaN,"Midbrain: 15.0%, Basal forebrain: 14.0%, Pons:...","Human SpC: 8.2%, Human CbDN: 7.6%, Human SN: 7.4%",...,0.022371,2184.314470,0.582884,0.021212,708.0,400.0,1364.0,2.0,0.551334,4.0
2,2.0,Nkcell_2,Miscellaneous,NK,NaN,NaN,NaN,NaN,"Thalamus: 17.6%, Cerebral cortex: 15.5%, Basal...","Human GPe: 5.7%, Human GPi: 5.5%, Human CbDN: ...",...,0.023788,2263.348733,0.558228,0.020093,245.0,126.0,300.0,0.0,0.447094,3.0


In [34]:
csv_columns = csv_df.columns.tolist()
exclude_columns = ["Cluster ID", "cell_label", "Supercluster"]
columns_to_ingest = [column for column in csv_columns if column not in exclude_columns]
columns_to_ingest

['Class auto-annotation',
 'Neurotransmitter auto-annotation',
 'Neuropeptide auto-annotation',
 'Subtype auto-annotation',
 'Transferred MTG Label',
 'Top three regions',
 'Top three dissections',
 'Top Enriched Genes',
 'Number of cells',
 'DoubletFinder score',
 'Total UMI',
 'Fraction unspliced',
 'Fraction mitochondrial',
 'H19.30.002',
 'H19.30.001',
 'H18.30.002',
 'H18.30.001',
 'Fraction cells from top donor',
 'Number of donors']

In [35]:
# cell_label is required, add it forcefully if not exist
for annotation in nn["annotations"]:
    if "cell_label" not in annotation:
        print(annotation)
        annotation["cell_label"] = "NA"

In [36]:
cellset_labels = [annotation["cell_label"] for annotation in neurons["annotations"]]
csv_df = csv_df[csv_df['cell_label'].isin(cellset_labels)]

In [38]:
updated_nn = add_author_annotations(nn, csv_df, "cell_label", columns_to_ingest)

ValueError: Extra keys in DataFrame that are not in CAS data: ['Bcell_0', 'Tcell_1', 'Nkcell_2', 'Mono_3', 'DLNP_83', 'DLCT6b_84', 'DLNP_85', 'DLNP_86', 'DLNP_87', 'DLNP_88', 'DLNP_89', 'DLNP_90', 'DLNP_91', 'DLNP_92', 'DLNP_93', 'DLNP_94', 'DLNP_95', 'DLNP_96', 'DLCT6b_97', 'DLCT6b_98', 'DLCT6b_99', 'DLCT6b_100', 'DLCT6b_101', 'DLCT6b_102', 'DLCT6b_103', 'DLCT6b_104', 'DLCT6b_105', 'DLCT6b_106', 'DLCT6b_107', 'DLCT6b_108', 'DLCT6b_109', 'DLCT6b_110', 'DLCT6b_111', 'DLCT6b_112', 'L5ET_113', 'L5ET_114', 'L5ET_115', 'Misc_116', 'L5ET_117', 'L5ET_118', 'CA13_119', 'ULIT_120', 'ULIT_121', 'ULIT_122', 'ULIT_123', 'ULIT_124', 'ULIT_125', 'ULIT_126', 'ULIT_127', 'ULIT_128', 'ULIT_129', 'ULIT_130', 'ULIT_131', 'Misc_132', 'ULIT_133', 'ULIT_134', 'ULIT_135', 'DLIT_136', 'DLIT_137', 'ULIT_138', 'DLIT_139', 'DLIT_140', 'DLIT_141', 'DLIT_142', 'DLIT_143', 'DLIT_144', 'DLIT_145', 'DLIT_146', 'DLIT_147', 'DLIT_148', 'DLIT_149', 'DLIT_150', 'DLIT_151', 'DLIT_152', 'Amex_153', 'Amex_154', 'Amex_155', 

In [ ]:
write_dict_to_json_file('./Final_json_files/Siletti_added_author_categories_v2_nn.json', updated_neurons)